# Scraping Portal da Transparência


### Autor: Wesley Wilson

<b>Informações do Mês 07/2022 (Última base disponibilizada pelo Governo Federal)</b><br>

### Inicio
 - Esse crawler retorna os dados do portal da Transparência, selecionando a última base de dados disponível.
 - Projeto desenvolvido em Python 3.10, foram utlizadas as bibliotecas selenium e BeautifulSoup, urllib, datetime, sqlalchemy e pandas além das nativas do python, os, time, datetime e csv.
 - As libs necessárias para o projeto encontram-se no requirements.txt

## Recursos utilizados no desenvolvimento:

- jupyter-lab
- python 3.10.5
- arquivo example_psql.py

### Importando Libs

* Importante: Se faz necessário o download do chromedriver de acordo com sua versão do navegador. 
* Download disponível em: https://chromedriver.chromium.org/downloads
* Foi usado o chromedriver da versão 102.0.5005.61

In [5]:
# Imports
import urllib.request as urllib_request
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from urllib.request import Request, urlopen
from urllib import request
from urllib.error import URLError, HTTPError
import os
import time
import datetime
import csv
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from zipfile import ZipFile
import example_psql as creds ##### Arquivo com dados usuario conexao Dbeaver
import numpy as np
from clint.textui import progress
import sys
import requests

#### Bot

In [7]:
# Create bot
class Bot():
    
    
    def __init__(self):
              
        # Webdriver
        options = webdriver.ChromeOptions()
        prefs = {"download.default_directory": r"directory",
                "directory_upgrade": True}
        options.add_experimental_option("prefs", prefs)
        driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
        # Connect DB Beaver
        engine = create_engine('postgresql://{}:{}@{}:5432/{}'.format(creds.PGUSER, creds.PGPASSWORD, creds.PGHOST, creds.PGDATABASE) , connect_args={'options': '-csearch_path={}'.format('data_science')})
        
        # Date and Files
        data = datetime.date.today()
        mes = int(data.month)-3
        option = str(mes)
        option = "0"+option
        option_mes = '//*[@id="links-meses"]/option['+ option +']'
        file_name = str(data.year)+option
        file = file_name+'.zip'
        
        # To get the current working directory
        directory = os.getcwd()
        
        # self
        self.GetData(driver, directory, data, mes, option, option_mes, file, file_name)
        self.ExportData(directory, file_name, engine)

        
    def GetData(self, driver, directory, data, mes, option, option_mes, file, file_name):
        # URL input
        driver.get("https://www.portaldatransparencia.gov.br/download-de-dados")
        print("Acessando Portal da Transparencia...")
    
        
        # Select benefit
        print("Selecionando o benefício...")
        time.sleep(5)
        
        # Benefits list
        beneficios = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btnAbaBeneficios"]'))).click()
        time.sleep(5)
        
        # Auxilio Brasil
        beneficio = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tableBeneficios"]/tbody/tr[1]/td[1]/a'))).click()
        
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.115 Safari/537.36'}

        print ("Coletando dados...")
        time.sleep(5)


        # Auxilio Brasil - Last month
        auxilio_brasil = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="links-meses"]/option['+ option +']'))).click()
        time.sleep(5)
        try:
            req = Request(driver.current_url, headers = headers)
            response = urlopen(req)
            #print(response.read())

        except HTTPError as e:
            print(e.status, e.reason)

        except URLError as e:
            print(e.reason)
        
        # Create variable URL
        url = driver.current_url
        file_url = url+'/'+file_name
        request.urlretrieve(file_url , file )
        time.sleep(5)
        print ("Download da base...")
        time.sleep(5)
        
        # Extract file
        print("Download concluído. Extraindo...")
        zf = ZipFile(file, 'r')
        zf.extractall(directory)
        zf.close()
       
    def ExportData(self, directory, file_name, engine):
        # Read CSV file
        print("Reading CSV file...")
        df = pd.read_csv(file_name+"_AuxilioBrasil.csv", encoding='iso-8859-1', 
                                warn_bad_lines=True, error_bad_lines=False, delimiter=';')
        
        # Export data frame to DBeaver
        print("Export data frame to DBeaver...")
        splits = np.array_split(df, 100)
        i = 0
        for split in splits :
            i +=1
            #print('start split ' + str(i) + ' ' + strftime("%Y-%m-%d %H:%M:%S", gmtime()))
            split.to_sql('auxilio_brasil', con=engine, if_exists='append', index=False)
            #print('stop split ' + str(i) + ' ' + strftime("%Y-%m-%d %H:%M:%S", gmtime()))
            print ('split ' + str(i) + ' inserido')
        
def main():
    My_BOt = Bot()

if __name__=='__main__':
    main()

C:\Users\WESLEYWILSONFERNANDE\AppData\Local\Temp\ipykernel_6576\1265152398.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
C:\Users\WESLEYWILSONFERNANDE\AppData\Local\Temp\ipykernel_6576\1265152398.py:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


Acessando Portal da Transparencia...
Selecionando o benefício...
Coletando dados...
Download da base...
Download concluído. Extraindo...
Reading CSV file...


C:\Users\WESLEYWILSONFERNANDE\AppData\Local\Temp\ipykernel_6576\1265152398.py:87: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_name+"_AuxilioBrasil.csv", encoding='iso-8859-1',
C:\Users\WESLEYWILSONFERNANDE\AppData\Local\Temp\ipykernel_6576\1265152398.py:87: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_name+"_AuxilioBrasil.csv", encoding='iso-8859-1',


Export data frame to DBeaver...
split 1 inserido
split 2 inserido
split 3 inserido
split 4 inserido
split 5 inserido
split 6 inserido
split 7 inserido
split 8 inserido
split 9 inserido
split 10 inserido
split 11 inserido
split 12 inserido
split 13 inserido
split 14 inserido
split 15 inserido
split 16 inserido
split 17 inserido
split 18 inserido
split 19 inserido
split 20 inserido
split 21 inserido
split 22 inserido
split 23 inserido
split 24 inserido
split 25 inserido
split 26 inserido
split 27 inserido
split 28 inserido
split 29 inserido
split 30 inserido
split 31 inserido
split 32 inserido
split 33 inserido
split 34 inserido
split 35 inserido
split 36 inserido
split 37 inserido
split 38 inserido
split 39 inserido
split 40 inserido
split 41 inserido
split 42 inserido
split 43 inserido
split 44 inserido
split 45 inserido
split 46 inserido
split 47 inserido
split 48 inserido
split 49 inserido
split 50 inserido
split 51 inserido
split 52 inserido
split 53 inserido
split 54 inserido
split